In [2]:
questions=["age","tipo","tecnica","family","history","symtomatic",
           "prosthesis","birads","density_mammo","calcifications_benign","density_echo","ganglio_mamo","lymph_benign","lymph_suspicious","parenchymal_distortion","simple_cyst","ductal_ectasia",
          "nodules_echo_num","nodules_echo_description","nodules_echo_shape","nodules_echo_margin","nodules_echo_echogenicity","nodules_echo_location","nodules_echo_size","nodules_echo_known","nodules_echo_stable"]



TIPO=["biopsy report", "nodal staging ultrasound report", "normal control or revision report"]
TECNICA=["only ultrasound study", "only mammography study", "mammography and ultrasound"]
FAMILY=["no family history", "first degree", "second degree"]
PROSTHESIS=["no prosthesis","yes prosthesis"]
BIRADS=["BI-RADS 0","BI-RADS 1","BI-RADS 2","BI-RADS 3","BI-RADS 4A","BI-RADS 4B","BI-RADS 4C","BI-RADS 5","BI-RADS 6"]
DENSITY_MAMMO=["ACR A","ACR B","ACR C","ACR D","unknown density mammo"]
CALCIFICATIONS_BENIGN=["no calcifications","yes calcifications"]
GANGLIO_MAMO=["no ganglio","yes ganglio"]
DENSITY_ECHO=["homogeneous fibroglandular","heterogeneous fibroglandular","fibroglandular and fat","homogeneous fatty","unknown density echo"]
LYMPH_BENIGN=["no lymph benign","yes lymph benign"]
LYMPH_SUSPICIOUS=["no lymph suspicious","yes lymph suspicious"]
SIMPLE_CYST=["no cyst","yes cyst"]
DUCTAL_ECTASIA=["no ectasia","yes ectasia"]
NODULES_ECHO=["no nodules", "yes nodules"]
NODULES_SHAPE=["oval","round","lobulated","irregular","unknown shape"]
NODULES_MARGIN=["circumscribed","spiculated","indistinct","not circumscribed","unknown margin"]
NODULES_ECHOGENICITY=["hypoechoic", "isoechoic", "heterogeneous","complex cystic and solid","unknown echogenicity"]
NODULES_KNOWN=["no known", "yes known"]
NODULES_STABLE=["grown stable","shrunk stable", "yes stable"]


import gc
from torch.nn.utils.rnn import pad_sequence
def flatten_and_filter_dataset(ground_truth,reports):
    """
    Esta función toma un conjunto de datos en el formato original (con estructura jerárquica)
    y devuelve un conjunto de datos plano, donde cada entrada tiene un solo `context`, `question` y `answer`.
    
    Argumentos:
        dataset: Un conjunto de datos en formato original (puede ser train, validation, test).
    
    Retorno:
        Un conjunto de datos de Hugging Face en formato plano, con solo ejemplos completos.
    """
    # Lista para almacenar ejemplos en formato plano
    flattened_examples = {}
    examples_raw={}
    targets={}
    val_data={}
    question_tipo={}
    previous_message_answer_tipo={}
    options_tipo={}
    answers_tipo={}
    j=0
    
    question_tipo["age"]= "does the patient's age appear in the following breast medical report?"
    previous_message_answer_tipo["age"]="search for numbers, but do not mistake it with the age of a familiar. If a number appears without any context between two dots it is surely the age."
    options_tipo["age"]="answer only the age of the patient."

    question_tipo["tipo"]= 'is the following breast medical report a biopsy report or a nodal staging ultrasound report?'
    previous_message_answer_tipo["tipo"]="biopsy reports are normally Image-Guided Biopsy and is normally said that they are referred to the hospital for biopsy. Nodal staging ultrasound reports can also be written as 'axilla ultrasound'. If it is any of these it will be written in the beginning of the report, normally in the used technique. These kind of reports are only ultrasound. In these reports no final BI-RADS is given."
    options_tipo["tipo"]="answer with one of the following options: 'biopsy report', 'nodal staging ultrasound report' or 'normal control or revision report'."

    question_tipo["tecnica"]= 'what diagnostic technique was used in the following breast medical report?'
    previous_message_answer_tipo["tecnica"]="biopsy reports, simple cysts and analysis of lymph or axillary nodes are only seen on ultrasound. On the other hand, if the ACR density is given or parenchymal distortions are analysed, the technique will be a mammogram. Tomosyntesis is a mammography type. The report may include an ultrasound examination, a mammography examination or both."
    options_tipo["tecnica"]="answer with one of the following options: 'only ultrasound study', 'only mammography study' or 'mammography and ultrasound'."

    question_tipo["family"]= "does the patient have any family history in the following breast medical report?"
    previous_message_answer_tipo["family"]="family history of breast cancer is categorized based on the degree of relatives affected: First-degree relatives: Parents, siblings, or children. Second-degree relatives: Grandparents, aunts, uncles, nieces, nephews, or half-siblings. Third-degree relatives: Great-grandparents, great-aunts/uncles, or first cousins."
    options_tipo["family"]="answer with one of the following options: 'first degree', 'second degree', 'third degree' or 'no family history'."
    
    question_tipo["history"]= "does the patient have any non-familiar history in the following breast medical report?"
    previous_message_answer_tipo["history"]="check for the history at the beginning of the report. Normally it is a previous biopsy result, mastectomy or cancer. It normally starts with 'history of ...'."
    options_tipo["history"]="answer retrieving the information directly from the report or with 'no history was found'."
    
    question_tipo["symtomatic"]= "is the reason for the consultation that the patient is symptomatic in the following breast medical report?"
    previous_message_answer_tipo["symtomatic"]="the answer is at the beginning of the report, in the reason for consultation. It is normally a palpable lump, lumpectomy or nodule, sometimes painful."
    options_tipo["symtomatic"]="answer retrieving the information directly from the report or with 'non-symptomatic consultation'."
    
    question_tipo["prosthesis"]= "does the patient have a prosthesis in the following breast medical report?"
    previous_message_answer_tipo["prosthesis"]="it is normally clearly indicated at the beginning of the report. Sometimes it is written as implants instead of prosthesis."
    options_tipo["prosthesis"]="answer with one of the following options: 'yes' or 'no'."
    
    question_tipo["birads"]= "what is the final BI-RADS classification given to the patient in the following breast medical report?"
    previous_message_answer_tipo["birads"]="the final BI-RADS of the patient is given in the conclusions of the report, normally at the end."
    options_tipo["birads"]="answer with one of the following options: 'BI-RADS 0', 'BI-RADS 1', 'BI-RADS 2', 'BI-RADS 3', 'BI-RADS 4A', 'BI-RADS 4B', 'BI-RADS 4C', 'BI-RADS 5' or 'unknown'."

    question_tipo["density_mammo"]= "what is the breast density found in the mammography study of the following breast medical report?"
    previous_message_answer_tipo["density_mammo"]="breast density in mammography is classified into four categories: ACR A (= Almost entirely fatty), ACR B (= Scattered areas of fibroglandular density), ACR C (= Heterogeneously dense), ACR D (= Extremely or very dense breasts). Sometimes it is written as 'density type x'. It can also be written with their real meaning (very dense breasts = C) and not with the A, B, C, D classification. Focus only on density."
    options_tipo["density_mammo"]="answer with one of the following options: 'ACR A', 'ACR B', 'ACR C', ACR D' or 'unknown'."
   
    question_tipo["density_echo"]= "what is the breast density found in the ultrasound study of the following breast medical report?"
    previous_message_answer_tipo["density_echo"]="breast composition in ultrasound is classified into three categories: fibroglandular and fat (mixed distribution of fibroglandular and adipose tissue), heterogeneous fibroglandular (predominantly fibroglandular tissue with varying echogenicity and scattered fat areas), homogeneous fatty (uniform fatty tissue with consistent echogenicity and minimal fibroglandular content), and homogeneous fibroglandular (uniform fibroglandular tissue with consistent echogenicity and minimal fat content)."
    options_tipo["density_echo"]="answer with one of the following options: 'fibroglandular and fat', 'heterogeneous fibroglandular', 'homogeneous fibroglandular', 'homogeneous fatty' or 'unknown'."

    question_tipo["calcifications_benign"]= "does the following breast medical report mention the appearence of benign calcifications in the mammography exam?"
    previous_message_answer_tipo["calcifications_benign"]="Consider only benign calcifications in the mammography."
    options_tipo["calcifications_benign"]="answer with one of the following options: 'yes' or 'no'."
    
    question_tipo["ganglio_mamo"]= "does the following breast medical report mention any lymph nodes in the mammography exam?"
    previous_message_answer_tipo["ganglio_mamo"]="Consider only lymph nodes that appear in the mammography."
    options_tipo["ganglio_mamo"]="answer with one of the following options: 'yes' or 'no'."

    question_tipo["parenchymal_distortion"]= "does the following breast medical report mention any parenchymal distortion or asymmetry in the mammography exam?"
    previous_message_answer_tipo["parenchymal_distortion"]="If it has any it will appear in the results of the mammography exam using the words distortion, asymmetry or sometimes it can also be surgical changes."
    options_tipo["parenchymal_distortion"]="answer retrieving the information directly from the report or with 'no'"
    
    question_tipo["lymph_suspicious"]= "does the following breast medical report mention any suspicious axillary lymph nodes in the ultrasound exam?"
    previous_message_answer_tipo["lymph_suspicious"]="if a lymph node is suspicious the report will recomend a biopsy or Fine Needle Aspiration. A lymph node is considered suspicious when it has eccentric cortical thickening ≥ 3 mm, Short axis >10 mm, round shape, loss of fatty hilum, abnormal vascularity, or irregular margins—especially when associated with known malignancy or progressive enlargement. They can also be classified as UN3, UN4 or UN5. An exam may have both suspicious and benign lymph nodes, answer 'yes' in this case."
    options_tipo["lymph_suspicious"]="answer with one of the following options: 'yes' or 'no'."

    question_tipo["lymph_benign"]= "does the following breast medical report mention any benign or not suspicious axillary lymph nodes in the ultrasound exam?"
    previous_message_answer_tipo["lymph_benign"]="if a lymph node is suspicious the report will recomend a biopsy or Fine Needle Aspiration. A lymph node is considered benign when it has uniform cortex < 3 mm, preserved fatty hilum, oval shape, no abnormal vascularity, no irregular margins and homogeneous internal echo pattern. Benign axillary nodes can be classified as UN1 or UN2. A reactive axillary node is not suspicious. An exam may have both suspicious and benign lymph nodes, answer 'yes' in this case."
    options_tipo["lymph_benign"]="answer with one of the following options: 'yes' or 'no'."

    question_tipo["simple_cyst"]= "does the following breast medical report mention any simple cysts or microcysts in the ultrasound exam?"
    previous_message_answer_tipo["simple_cyst"]="The words symple cysts or microcysts will appear only in the ultrasound exam. Sometimes they can say that some of the cysts have echogenic content, but we still will consider them simple cysts and not nodules."
    options_tipo["simple_cyst"]="answer with one of the following options: 'yes' or 'no'."

    question_tipo["ductal_ectasia"]= "does the following breast medical report mention any ductal ectasia in the ultrasound exam?"
    previous_message_answer_tipo["ductal_ectasia"]="The word ductal ectasia will appear only in the ultrasound exam."
    options_tipo["ductal_ectasia"]="answer with one of the following options: 'yes' or 'no'."
    
    question_tipo["nodules_echo_num"]="is there any nodule described in the ultrasound exam of the following breast medical report?"
    previous_message_answer_tipo["nodules_echo_num"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. The localization, echogenicity and size of the nodules are normally said."
    options_tipo["nodules_echo_num"]="answer with one of the following options: 'yes' or 'no'."

    
    dic_order = {
        1: "first",
        2: "second",
        3: "third",
        4: "fourth",
        5: "fifth",
        6: "sixth",
        7: "seventh",
        8: "eighth",
        9: "ninth",
        10: "tenth",
        11: "eleventh",
        12: "twelfth",
        13: "thirteenth",
        14: "fourteenth",
        15: "fifteenth"
    }
    for i in range(1,2):
        question_tipo[f"nodules_echo_description_{i}"]= f"which is the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_description_{i}"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_description_{i}"]="answer retrieving the information directly from the report."
        
        question_tipo[f"nodules_echo_shape_{i}"]= f"what is the shape of the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_shape_{i}"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. Shapes can be 'oval', 'round', 'lobulated' and 'irregular'. Sometimes irregular is also used for the margin, but in this case it is written as 'irregular margin' or 'irregular borders'. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_shape_{i}"]="answer with one of the following options: 'oval', 'round', 'lobulated', 'irregular' or 'unknown'."
        
        question_tipo[f"nodules_echo_margin_{i}"]= f"what is the margin of the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_margin_{i}"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. Margin can be 'circumscribed' and 'not circumscribed'. Inside the not circumscribed we have 'spiculated', 'angulated', 'microlobulated' or 'indistinc' ('not defined') margins. Sometimes irregular is also used for the margin, but in this case it is written as 'irregular margin' or 'irregular borders', in this case classify it as 'not circumscribed'. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_margin_{i}"]="answer with one of the following options: 'circumscribed', 'not circumscribed', 'indefined', 'spiculated', 'angulated', 'microlobulated' or 'unknown'."
        
        question_tipo[f"nodules_echo_echogenicity_{i}"]= f"what is the echogenicity of the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_echogenicity_{i}"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. Echogenicity can be 'anechoic', 'hypoechoic', 'heterogeneous' and 'complex cystic and solid'. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour." 
        options_tipo[f"nodules_echo_echogenicity_{i}"]="answer with one of the following options: 'hypoechoic', 'heterogeneous', 'anechoic', 'hyperecoic', 'isoechoic', 'complex cystic and solid' or 'unknown'."
        
        question_tipo[f"nodules_echo_location_{i}"]= f"In which location is the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_location_{i}"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. If the nodule is mentioned previously in the mammography, the location can be found also there. sometimes the breast location of the tumour may be written in a different part than the quadrant. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_location_{i}"]="answer retrieving the information directly from the report or with 'unknown'."
        
        question_tipo[f"nodules_echo_size_{i}"]= f"what is the size of the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_size_{i}"]="do not consider if a nodule is described in the mammography exam. The localization, echogenicity and size of the nodules are normally said. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_size_{i}"]="answer retrieving the information directly from the report (stop after 'mm') or with 'unknown'."
        
        question_tipo[f"nodules_echo_known_{i}"]= f"is the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report previously known?"
        previous_message_answer_tipo[f"nodules_echo_known_{i}"]="do not consider if a nodule is described in the mammography exam. If the nodule is known from before the report, it will say if it it is stable or if it has grown or shrink. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_known_{i}"]="answer with one of the following options: 'yes' or 'no'."
        
        question_tipo[f"nodules_echo_stable_{i}"]= f"is the {dic_order[i]} known nodule described in the ultrasound exam stable in the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_stable_{i}"]="do not consider if a nodule is described in the mammography exam. If the nodule is known from before the examination, it will be analysed to see if it is stable or if it got bigger or smaller. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_stable_{i}"]="answer with one of the following options: 'yes', 'grown' or 'shrunk."
    for i, report in enumerate(reports["informes_ingles"]):
        informe=preprocess_text(report)
        key=reports["keys"][i]
        
        if key not in ground_truth.index:
            continue
        
        row=ground_truth.loc[key]

        #AGE
        age=str(row["Age"])
        answer_tipo={}
        # Verificar si el ejemplo tiene preguntas
        if age.isdigit():
            answer_tipo["age"]=age+"."
            
        else:
            answer_tipo["age"]="no"+"."
        
        #TIPO
        if row["Biopsy_report"]=="Yes":
            answer_tipo["tipo"]="biopsy report"+"."
            
        elif row["Ganglio_report"]=="Yes":
            answer_tipo["tipo"]="nodal staging ultrasound report"+"."
        else:
            answer_tipo["tipo"]="normal control or revision report"+"."
        
        #TECHNIQUE
        tecnica=row["Technique"]
        # Verificar si el ejemplo tiene preguntas
        if tecnica=="ultrasound":
            answer_tipo["tecnica"]="only ultrasound study"+"."            
        elif tecnica=="mammography":
            answer_tipo["tecnica"]="only mammography study"+"."
        elif not pd.isna(tecnica):
            answer_tipo["tecnica"]=tecnica+"."
        else:
            print(key,report)
        
        # 
        # HISTORY
        #No consideramos las biopsias o las ecografías de estadificación ganglionar.
        if answer_tipo["tipo"]=="normal control or revision report"+".":
            history=row["Other_history"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(history,str) or history=="No":
                answer_tipo["history"]="no history was found"+"."            
            else:
                answer_tipo["history"]=history+"." 
    
            # FAMILY
            family=row["Family_history"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(family,str) or family=="No":
                answer_tipo["family"]="no family history"+"."            
            else:
                answer_tipo["family"]=family+"." 
    
            # SYMTOMATIC
            symtomatic=row["Syntomatic"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(symtomatic,str) or symtomatic=="No" or symtomatic=="No estoy seguro":
                answer_tipo["symtomatic"]="Non-symptomatic consultation"+"."            
            else:
                answer_tipo["symtomatic"]=symtomatic+"." 
    
            # PROSTHESIS
            prosthesis=row["Prosthesis"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(prosthesis,str) or prosthesis.lower()=="no":
                answer_tipo["prosthesis"]="no"+"."            
            else:
                answer_tipo["prosthesis"]="yes"+"."
    
            #BIRADS
            birads=row["BI-RADS"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(birads,str):
                answer_tipo["birads"]="unknown"+"."            
            else:
                answer_tipo["birads"]=birads+"."
    
            #Density mammo
            density_mammo=row["Density_mamo"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(density_mammo,str):
                answer_tipo["density_mammo"]="unknown"+"."            
            else:
                answer_tipo["density_mammo"]=density_mammo+"."

            #Lymp nodes mammo
            ganglio_mamo=row["Ganglio_mamo"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(ganglio_mamo,str):
                answer_tipo["ganglio_mamo"]="no"+"."            
            else:
                answer_tipo["ganglio_mamo"]=ganglio_mamo.lower()+"."

            #Calcifications benign
            calcifications_benign=row["Calcifications_benign_mamo"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(calcifications_benign,str):
                answer_tipo["calcifications_benign"]="no"+"."            
            else:
                answer_tipo["calcifications_benign"]=calcifications_benign.lower()+"."

            #Parenchymal distortion
            parenchymal_distortion=row["parenchymal_distortions_asymmetry"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(parenchymal_distortion,str):
                answer_tipo["parenchymal_distortion"]="no"+"."            
            else:
                answer_tipo["parenchymal_distortion"]=parenchymal_distortion.lower()+"."
    
            #Density echo
            density_echo=row["Density_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(density_echo,str):
                answer_tipo["density_echo"]="unknown"+"."            
            else:
                if density_echo in DENSITY_ECHO:
                    answer_tipo["density_echo"]=density_echo+"."

            #Benign lymph nodes
            simple_cyst=row["simple_cyst_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(simple_cyst,str):
                answer_tipo["simple_cyst"]="no"+"."            
            else:
                answer_tipo["simple_cyst"]=simple_cyst.lower()+"."
            #Suspicious lymph nodes
            lymph_suspicious=row["Ganglio_suspicious_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(lymph_suspicious,str):
                answer_tipo["lymph_suspicious"]="no"+"."            
            else:
                answer_tipo["lymph_suspicious"]=lymph_suspicious.lower()+"."

            #Benign lymph nodes
            lymph_benign=row["Ganglio_benign_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(lymph_benign,str):
                answer_tipo["lymph_benign"]="no"+"."            
            else:
                answer_tipo["lymph_benign"]=lymph_benign.lower()+"."

            #Ductal ectasia
            ductal_ectasia=row["Ductal_ectasia_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(ductal_ectasia,str):
                answer_tipo["ductal_ectasia"]="no"+"."            
            else:
                answer_tipo["ductal_ectasia"]=ductal_ectasia.lower()+"."
    
            #Nodules echo
            nodules_echo_num=row["Nodules_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(nodules_echo_num,str) and not isinstance(nodules_echo_num,int):
                answer_tipo["nodules_echo_num"]="no"+"."    
            elif isinstance(nodules_echo_num,str) and nodules_echo_num.lower()=="no":
                answer_tipo["nodules_echo_num"]="no"+"."  
            else:
                answer_tipo["nodules_echo_num"]="yes"+"."
    
            #Si existen nódulos se hace las preguntas correspondientes
            if answer_tipo["nodules_echo_num"]!="no.":
                nodules_echo_description=row["Description_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                
                answer_tipo["nodules_echo_description_1"]=nodules_echo_description+"."
                    
                nodules_echo_shape=row[f"Shape_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_shape,str):
                    answer_tipo["nodules_echo_shape_1"]="unknown"+"."
                else:
                    answer_tipo["nodules_echo_shape_1"]=nodules_echo_shape.lower()+"."

                nodules_echo_margin=row["Margin_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_margin,str):
                    answer_tipo["nodules_echo_margin_1"]="unknown"+"."
                else:
                    answer_tipo["nodules_echo_margin_1"]=nodules_echo_margin.lower()+"."

                nodules_echo_echogenicity=row["Echogenicity_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_echogenicity,str):
                    answer_tipo["nodules_echo_echogenicity_1"]="unknown"+"."
                else:
                    answer_tipo["nodules_echo_echogenicity_1"]=nodules_echo_echogenicity.lower()+"."

                nodules_echo_location=row[f"Location_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_location,str):
                    answer_tipo["nodules_echo_location_1"]="unknown"+"."
                else:
                    answer_tipo["nodules_echo_location_1"]=nodules_echo_location.lower()+"."

                
                #Nodules echo size
                nodules_echo_size=row["size_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_size,str):
                    answer_tipo["nodules_echo_size_1"]="unknown"+"."            
                else:
                    answer_tipo["nodules_echo_size_1"]=nodules_echo_size+"."
        
                #Nodules echo known
                nodules_echo_known=row["new_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_known,str):
                    answer_tipo["nodules_echo_known_1"]="unknown"+"."
                elif nodules_echo_known.lower()=="no":
                    answer_tipo["nodules_echo_known_1"]="yes"+"."            
                else:
                    answer_tipo["nodules_echo_known_1"]="no"+"."
    
                if answer_tipo["nodules_echo_known_1"]=="yes.":
                    #Nodules echo stable
                    nodules_echo_stable=row["Stable_eco_1"]
                    # Verificar si el ejemplo tiene preguntas
                    if not isinstance(nodules_echo_stable,str):
                        answer_tipo["nodules_echo_stable_1"]="unknown"+"."
                    else:
                        answer_tipo["nodules_echo_stable_1"]=nodules_echo_stable.lower()+"."
            
        for tipo in answer_tipo:
            #Si está el tipo en las respuestas que hemos recogido lo metemos a la base de datos.
            key_tipo=key+"_"+tipo
            if key_tipo in flattened_examples:
                continue
                key_tipo=key_tipo+"_copy"
            examples_raw[key_tipo]=report
            answer=str(answer_tipo[tipo])
            
    
            inputs_tipo = "Question: " + question_tipo[tipo] +" Additional information: "+ previous_message_answer_tipo[tipo]+ " Context: " + informe + "Options:"+ options_tipo[tipo]+" Answer: "+ str(answer_tipo[tipo])
            flattened_examples[key_tipo]=inputs_tipo
            targets[key_tipo]=answer
            
            val_data[key_tipo]="Question: " + question_tipo[tipo] +" Additional information: "+ previous_message_answer_tipo[tipo]+ " Context: " + informe+ "Options:"+ options_tipo[tipo] + " Answer: "
    return flattened_examples,targets,val_data,examples_raw

def flatten_and_filter_dataset_no_prev(ground_truth,reports):
    """
    Esta función toma un conjunto de datos en el formato original (con estructura jerárquica)
    y devuelve un conjunto de datos plano, donde cada entrada tiene un solo `context`, `question` y `answer`.
    
    Argumentos:
        dataset: Un conjunto de datos en formato original (puede ser train, validation, test).
    
    Retorno:
        Un conjunto de datos de Hugging Face en formato plano, con solo ejemplos completos.
    """
    """
    Esta función toma un conjunto de datos en el formato original (con estructura jerárquica)
    y devuelve un conjunto de datos plano, donde cada entrada tiene un solo `context`, `question` y `answer`.
    
    Argumentos:
        dataset: Un conjunto de datos en formato original (puede ser train, validation, test).
    
    Retorno:
        Un conjunto de datos de Hugging Face en formato plano, con solo ejemplos completos.
    """
    # Lista para almacenar ejemplos en formato plano
    flattened_examples = {}
    examples_raw={}
    targets={}
    val_data={}
    question_tipo={}
    previous_message_answer_tipo={}
    options_tipo={}
    answers_tipo={}
    j=0
    
    question_tipo["age"]= "does the patient's age appear in the following breast medical report?"
    previous_message_answer_tipo["age"]="search for numbers, but do not mistake it with the age of a familiar. If a number appears without any context between two dots it is surely the age."
    options_tipo["age"]="answer only the age of the patient."

    question_tipo["tipo"]= 'is the following breast medical report a biopsy report or a nodal staging ultrasound report?'
    previous_message_answer_tipo["tipo"]="biopsy reports are normally Image-Guided Biopsy and is normally said that they are referred to the hospital for biopsy. Nodal staging ultrasound reports can also be written as 'axilla ultrasound'. If it is any of these it will be written in the beginning of the report, normally in the used technique. These kind of reports are only ultrasound. In these reports no final BI-RADS is given."
    options_tipo["tipo"]="answer with one of the following options: 'biopsy report', 'nodal staging ultrasound report' or 'normal control or revision report'."

    question_tipo["tecnica"]= 'what diagnostic technique was used in the following breast medical report?'
    previous_message_answer_tipo["tecnica"]="biopsy reports, simple cysts and analysis of lymph or axillary nodes are only seen on ultrasound. On the other hand, if the ACR density is given or parenchymal distortions are analysed, the technique will be a mammogram. Tomosyntesis is a mammography type. The report may include an ultrasound examination, a mammography examination or both."
    options_tipo["tecnica"]="answer with one of the following options: 'only ultrasound study', 'only mammography study' or 'mammography and ultrasound'."

    question_tipo["family"]= "does the patient have any family history in the following breast medical report?"
    previous_message_answer_tipo["family"]="family history of breast cancer is categorized based on the degree of relatives affected: First-degree relatives: Parents, siblings, or children. Second-degree relatives: Grandparents, aunts, uncles, nieces, nephews, or half-siblings. Third-degree relatives: Great-grandparents, great-aunts/uncles, or first cousins."
    options_tipo["family"]="answer with one of the following options: 'first degree', 'second degree', 'third degree' or 'no family history'."
    
    question_tipo["history"]= "does the patient have any non-familiar history in the following breast medical report?"
    previous_message_answer_tipo["history"]="check for the history at the beginning of the report. Normally it is a previous biopsy result, mastectomy or cancer. It normally starts with 'history of ...'."
    options_tipo["history"]="answer retrieving the information directly from the report or with 'no history was found'."
    
    question_tipo["symtomatic"]= "is the reason for the consultation that the patient is symptomatic in the following breast medical report?"
    previous_message_answer_tipo["symtomatic"]="the answer is at the beginning of the report, in the reason for consultation. It is normally a palpable lump, lumpectomy or nodule, sometimes painful."
    options_tipo["symtomatic"]="answer retrieving the information directly from the report or with 'non-symptomatic consultation'."
    
    question_tipo["prosthesis"]= "does the patient have a prosthesis in the following breast medical report?"
    previous_message_answer_tipo["prosthesis"]="it is normally clearly indicated at the beginning of the report. Sometimes it is written as implants instead of prosthesis."
    options_tipo["prosthesis"]="answer with one of the following options: 'yes' or 'no'."
    
    question_tipo["birads"]= "what is the final BI-RADS classification given to the patient in the following breast medical report?"
    previous_message_answer_tipo["birads"]="the final BI-RADS of the patient is given in the conclusions of the report, normally at the end."
    options_tipo["birads"]="answer with one of the following options: 'BI-RADS 0', 'BI-RADS 1', 'BI-RADS 2', 'BI-RADS 3', 'BI-RADS 4A', 'BI-RADS 4B', 'BI-RADS 4C', 'BI-RADS 5' or 'unknown'."

    question_tipo["density_mammo"]= "what is the breast density found in the mammography study of the following breast medical report?"
    previous_message_answer_tipo["density_mammo"]="breast density in mammography is classified into four categories: ACR A (= Almost entirely fatty), ACR B (= Scattered areas of fibroglandular density), ACR C (= Heterogeneously dense), ACR D (= Extremely or very dense breasts). Sometimes it is written as 'density type x'. It can also be written with their real meaning (very dense breasts = C) and not with the A, B, C, D classification. Focus only on density."
    options_tipo["density_mammo"]="answer with one of the following options: 'ACR A', 'ACR B', 'ACR C', ACR D' or 'unknown'."
   
    question_tipo["density_echo"]= "what is the breast density found in the ultrasound study of the following breast medical report?"
    previous_message_answer_tipo["density_echo"]="breast composition in ultrasound is classified into three categories: fibroglandular and fat (mixed distribution of fibroglandular and adipose tissue), heterogeneous fibroglandular (predominantly fibroglandular tissue with varying echogenicity and scattered fat areas), homogeneous fatty (uniform fatty tissue with consistent echogenicity and minimal fibroglandular content), and homogeneous fibroglandular (uniform fibroglandular tissue with consistent echogenicity and minimal fat content)."
    options_tipo["density_echo"]="answer with one of the following options: 'fibroglandular and fat', 'heterogeneous fibroglandular', 'homogeneous fibroglandular', 'homogeneous fatty' or 'unknown'."

    question_tipo["calcifications_benign"]= "does the following breast medical report mention the appearence of benign calcifications in the mammography exam?"
    previous_message_answer_tipo["calcifications_benign"]="Consider only benign calcifications in the mammography."
    options_tipo["calcifications_benign"]="answer with one of the following options: 'yes' or 'no'."
    
    question_tipo["ganglio_mamo"]= "does the following breast medical report mention any lymph nodes in the mammography exam?"
    previous_message_answer_tipo["ganglio_mamo"]="Consider only lymph nodes that appear in the mammography."
    options_tipo["ganglio_mamo"]="answer with one of the following options: 'yes' or 'no'."

    question_tipo["parenchymal_distortion"]= "does the following breast medical report mention any parenchymal distortion or asymmetry in the mammography exam?"
    previous_message_answer_tipo["parenchymal_distortion"]="If it has any it will appear in the results of the mammography exam using the words distortion, asymmetry or sometimes it can also be surgical changes."
    options_tipo["parenchymal_distortion"]="answer retrieving the information directly from the report or with 'no'"
    
    question_tipo["lymph_suspicious"]= "does the following breast medical report mention any suspicious axillary lymph nodes in the ultrasound exam?"
    previous_message_answer_tipo["lymph_suspicious"]="if a lymph node is suspicious the report will recomend a biopsy or Fine Needle Aspiration. A lymph node is considered suspicious when it has eccentric cortical thickening ≥ 3 mm, Short axis >10 mm, round shape, loss of fatty hilum, abnormal vascularity, or irregular margins—especially when associated with known malignancy or progressive enlargement. They can also be classified as UN3, UN4 or UN5. An exam may have both suspicious and benign lymph nodes, answer 'yes' in this case."
    options_tipo["lymph_suspicious"]="answer with one of the following options: 'yes' or 'no'."

    question_tipo["lymph_benign"]= "does the following breast medical report mention any benign or not suspicious axillary lymph nodes in the ultrasound exam?"
    previous_message_answer_tipo["lymph_benign"]="if a lymph node is suspicious the report will recomend a biopsy or Fine Needle Aspiration. A lymph node is considered benign when it has uniform cortex < 3 mm, preserved fatty hilum, oval shape, no abnormal vascularity, no irregular margins and homogeneous internal echo pattern. Benign axillary nodes can be classified as UN1 or UN2. A reactive axillary node is not suspicious. An exam may have both suspicious and benign lymph nodes, answer 'yes' in this case."
    options_tipo["lymph_benign"]="answer with one of the following options: 'yes' or 'no'."

    question_tipo["simple_cyst"]= "does the following breast medical report mention any simple cysts or microcysts in the ultrasound exam?"
    previous_message_answer_tipo["simple_cyst"]="The words symple cysts or microcysts will appear only in the ultrasound exam. Sometimes they can say that some of the cysts have echogenic content, but we still will consider them simple cysts and not nodules."
    options_tipo["simple_cyst"]="answer with one of the following options: 'yes' or 'no'."

    question_tipo["ductal_ectasia"]= "does the following breast medical report mention any ductal ectasia in the ultrasound exam?"
    previous_message_answer_tipo["ductal_ectasia"]="The word ductal ectasia will appear only in the ultrasound exam."
    options_tipo["ductal_ectasia"]="answer with one of the following options: 'yes' or 'no'."
    
    question_tipo["nodules_echo_num"]="is there any nodule described in the ultrasound exam of the following breast medical report?"
    previous_message_answer_tipo["nodules_echo_num"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. The localization, echogenicity and size of the nodules are normally said."
    options_tipo["nodules_echo_num"]="answer with one of the following options: 'yes' or 'no'."

    
    dic_order = {
        1: "first",
        2: "second",
        3: "third",
        4: "fourth",
        5: "fifth",
        6: "sixth",
        7: "seventh",
        8: "eighth",
        9: "ninth",
        10: "tenth",
        11: "eleventh",
        12: "twelfth",
        13: "thirteenth",
        14: "fourteenth",
        15: "fifteenth"
    }
    for i in range(1,2):
        question_tipo[f"nodules_echo_description_{i}"]= f"which is the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_description_{i}"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_description_{i}"]="answer retrieving the information directly from the report."
        
        question_tipo[f"nodules_echo_shape_{i}"]= f"what is the shape of the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_shape_{i}"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. Shapes can be 'oval', 'round', 'lobulated' and 'irregular'. Sometimes irregular is also used for the margin, but in this case it is written as 'irregular margin' or 'irregular borders'. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_shape_{i}"]="answer with one of the following options: 'oval', 'round', 'lobulated', 'irregular' or 'unknown'."
        
        question_tipo[f"nodules_echo_margin_{i}"]= f"what is the margin of the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_margin_{i}"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. Margin can be 'circumscribed' and 'not circumscribed'. Inside the not circumscribed we have 'spiculated', 'angulated', 'microlobulated' or 'indistinc' ('not defined') margins. Sometimes irregular is also used for the margin, but in this case it is written as 'irregular margin' or 'irregular borders', in this case classify it as 'not circumscribed'. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_margin_{i}"]="answer with one of the following options: 'circumscribed', 'not circumscribed', 'indefined', 'spiculated', 'angulated', 'microlobulated' or 'unknown'."
        
        question_tipo[f"nodules_echo_echogenicity_{i}"]= f"what is the echogenicity of the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_echogenicity_{i}"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. Echogenicity can be 'anechoic', 'hypoechoic', 'heterogeneous' and 'complex cystic and solid'. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour." 
        options_tipo[f"nodules_echo_echogenicity_{i}"]="answer with one of the following options: 'hypoechoic', 'heterogeneous', 'anechoic', 'hyperecoic', 'isoechoic', 'complex cystic and solid' or 'unknown'."
        
        question_tipo[f"nodules_echo_location_{i}"]= f"In which location is the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_location_{i}"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. If the nodule is mentioned previously in the mammography, the location can be found also there. sometimes the breast location of the tumour may be written in a different part than the quadrant. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_location_{i}"]="answer retrieving the information directly from the report or with 'unknown'."
        
        question_tipo[f"nodules_echo_size_{i}"]= f"what is the size of the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_size_{i}"]="do not consider if a nodule is described in the mammography exam. The localization, echogenicity and size of the nodules are normally said. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_size_{i}"]="answer retrieving the information directly from the report (stop after 'mm') or with 'unknown'."
        
        question_tipo[f"nodules_echo_known_{i}"]= f"is the {dic_order[i]} nodule described in the ultrasound exam of the following breast medical report previously known?"
        previous_message_answer_tipo[f"nodules_echo_known_{i}"]="do not consider if a nodule is described in the mammography exam. If the nodule is known from before the report, it will say if it it is stable or if it has grown or shrink. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_known_{i}"]="answer with one of the following options: 'yes' or 'no'."
        
        question_tipo[f"nodules_echo_stable_{i}"]= f"is the {dic_order[i]} known nodule described in the ultrasound exam stable in the following breast medical report?"
        previous_message_answer_tipo[f"nodules_echo_stable_{i}"]="do not consider if a nodule is described in the mammography exam. If the nodule is known from before the examination, it will be analysed to see if it is stable or if it got bigger or smaller. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
        options_tipo[f"nodules_echo_stable_{i}"]="answer with one of the following options: 'yes', 'grown' or 'shrunk."
    for i, report in enumerate(reports["informes_ingles"]):
        informe=preprocess_text(report)
        key=reports["keys"][i]
        
        if key not in ground_truth.index:
            continue
        
        row=ground_truth.loc[key]

        #AGE
        age=str(row["Age"])
        answer_tipo={}
        # Verificar si el ejemplo tiene preguntas
        if age.isdigit():
            answer_tipo["age"]=age+"."
            
        else:
            answer_tipo["age"]="no"+"."
        
        #TIPO
        if row["Biopsy_report"]=="Yes":
            answer_tipo["tipo"]="biopsy report"+"."
            
        elif row["Ganglio_report"]=="Yes":
            answer_tipo["tipo"]="nodal staging ultrasound report"+"."
        else:
            answer_tipo["tipo"]="normal control or revision report"+"."
        
        #TECHNIQUE
        tecnica=row["Technique"]
        # Verificar si el ejemplo tiene preguntas
        if tecnica=="ultrasound":
            answer_tipo["tecnica"]="only ultrasound study"+"."            
        elif tecnica=="mammography":
            answer_tipo["tecnica"]="only mammography study"+"."
        elif not pd.isna(tecnica):
            answer_tipo["tecnica"]=tecnica+"."
        else:
            print(key,report)
        
        # 
        # HISTORY
        #No consideramos las biopsias o las ecografías de estadificación ganglionar.
        if answer_tipo["tipo"]=="normal control or revision report"+".":
            history=row["Other_history"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(history,str) or history=="No":
                answer_tipo["history"]="no history was found"+"."            
            else:
                answer_tipo["history"]=history+"." 
    
            # FAMILY
            family=row["Family_history"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(family,str) or family=="No":
                answer_tipo["family"]="no family history"+"."            
            else:
                answer_tipo["family"]=family+"." 
    
            # SYMTOMATIC
            symtomatic=row["Syntomatic"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(symtomatic,str) or symtomatic=="No" or symtomatic=="No estoy seguro":
                answer_tipo["symtomatic"]="Non-symptomatic consultation"+"."            
            else:
                answer_tipo["symtomatic"]=symtomatic+"." 
    
            # PROSTHESIS
            prosthesis=row["Prosthesis"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(prosthesis,str) or prosthesis=="No":
                answer_tipo["prosthesis"]="no"+"."            
            else:
                answer_tipo["prosthesis"]="yes"+"."
    
            #BIRADS
            birads=row["BI-RADS"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(birads,str):
                answer_tipo["birads"]="unknown"+"."            
            else:
                answer_tipo["birads"]=birads+"."
    
            #Density mammo
            density_mammo=row["Density_mamo"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(density_mammo,str):
                answer_tipo["density_mammo"]="unknown"+"."            
            else:
                answer_tipo["density_mammo"]=density_mammo+"."

            #Lymp nodes mammo
            ganglio_mamo=row["Ganglio_mamo"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(ganglio_mamo,str):
                answer_tipo["ganglio_mamo"]="no"+"."            
            else:
                answer_tipo["ganglio_mamo"]=ganglio_mamo.lower()+"."

            #Calcifications benign
            calcifications_benign=row["Calcifications_benign_mamo"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(calcifications_benign,str):
                answer_tipo["calcifications_benign"]="no"+"."            
            else:
                answer_tipo["calcifications_benign"]=calcifications_benign.lower()+"."

            #Parenchymal distortion
            parenchymal_distortion=row["parenchymal_distortions_asymmetry"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(parenchymal_distortion,str):
                answer_tipo["parenchymal_distortion"]="no"+"."            
            else:
                answer_tipo["parenchymal_distortion"]=parenchymal_distortion.lower()+"."
    
            #Density echo
            density_echo=row["Density_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(density_echo,str):
                answer_tipo["density_echo"]="unknown"+"."            
            else:
                if density_echo in DENSITY_ECHO:
                    answer_tipo["density_echo"]=density_echo+"."

            #Benign lymph nodes
            simple_cyst=row["simple_cyst_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(simple_cyst,str):
                answer_tipo["simple_cyst"]="no"+"."            
            else:
                answer_tipo["simple_cyst"]=simple_cyst.lower()+"."
            #Suspicious lymph nodes
            lymph_suspicious=row["Ganglio_suspicious_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(lymph_suspicious,str):
                answer_tipo["lymph_suspicious"]="no"+"."            
            else:
                answer_tipo["lymph_suspicious"]=lymph_suspicious.lower()+"."

            #Benign lymph nodes
            lymph_benign=row["Ganglio_benign_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(lymph_benign,str):
                answer_tipo["lymph_benign"]="no"+"."            
            else:
                answer_tipo["lymph_benign"]=lymph_benign.lower()+"."

            #Ductal ectasia
            ductal_ectasia=row["Ductal_ectasia_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(ductal_ectasia,str):
                answer_tipo["ductal_ectasia"]="no"+"."            
            else:
                answer_tipo["ductal_ectasia"]=ductal_ectasia.lower()+"."
    
            #Nodules echo
            nodules_echo_num=row["Nodules_eco"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(nodules_echo_num,str) and not isinstance(nodules_echo_num,int):
                answer_tipo["nodules_echo_num"]="no"+"."    
            elif isinstance(nodules_echo_num,str) and nodules_echo_num=="No":
                answer_tipo["nodules_echo_num"]="no"+"."  
            else:
                answer_tipo["nodules_echo_num"]="yes"+"."
    
            #Si existen nódulos se hace las preguntas correspondientes
            if answer_tipo["nodules_echo_num"]!="no.":
                nodules_echo_description=row["Description_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                
                answer_tipo["nodules_echo_description_1"]=nodules_echo_description+"."
                    
                nodules_echo_shape=row[f"Shape_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_shape,str):
                    answer_tipo["nodules_echo_shape_1"]="unknown"+"."
                else:
                    answer_tipo["nodules_echo_shape_1"]=nodules_echo_shape.lower()+"."

                nodules_echo_margin=row["Margin_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_margin,str):
                    answer_tipo["nodules_echo_margin_1"]="unknown"+"."
                else:
                    answer_tipo["nodules_echo_margin_1"]=nodules_echo_margin.lower()+"."

                nodules_echo_echogenicity=row["Echogenicity_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_echogenicity,str):
                    answer_tipo["nodules_echo_echogenicity_1"]="unknown"+"."
                else:
                    answer_tipo["nodules_echo_echogenicity_1"]=nodules_echo_echogenicity.lower()+"."

                nodules_echo_location=row[f"Location_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_location,str):
                    answer_tipo["nodules_echo_location_1"]="unknown"+"."
                else:
                    answer_tipo["nodules_echo_location_1"]=nodules_echo_location.lower()+"."

                
                #Nodules echo size
                nodules_echo_size=row["size_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_size,str):
                    answer_tipo["nodules_echo_size_1"]="unknown"+"."            
                else:
                    answer_tipo["nodules_echo_size_1"]=nodules_echo_size+"."
        
                #Nodules echo known
                nodules_echo_known=row["new_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_known,str):
                    answer_tipo["nodules_echo_known_1"]="unknown"+"."
                elif nodules_echo_known=="No":
                    answer_tipo["nodules_echo_known_1"]="yes"+"."            
                else:
                    answer_tipo["nodules_echo_known_1"]="no"+"."
    
                if answer_tipo["nodules_echo_known_1"]=="yes.":
                    #Nodules echo stable
                    nodules_echo_stable=row["Stable_eco_1"]
                    # Verificar si el ejemplo tiene preguntas
                    if not isinstance(nodules_echo_stable,str):
                        answer_tipo["nodules_echo_stable_1"]="unknown"+"."
                    else:
                        answer_tipo["nodules_echo_stable_1"]=nodules_echo_stable.lower()+"."
            
        for tipo in answer_tipo:
            #Si está el tipo en las respuestas que hemos recogido lo metemos a la base de datos.
            key_tipo=key+"_"+tipo
            if key_tipo in flattened_examples:
                continue
                key_tipo=key_tipo+"_copy"
            examples_raw[key_tipo]=report
            answer=str(answer_tipo[tipo])
            
    
            inputs_tipo = "Question: " + question_tipo[tipo]+ " Context: " + informe +" Answer: "+ str(answer_tipo[tipo])
            flattened_examples[key_tipo]=inputs_tipo
            targets[key_tipo]=answer
            
            val_data[key_tipo]="Question: " + question_tipo[tipo] + " Context: " + informe+ " Answer: "
    return flattened_examples,targets,val_data,examples_raw

    
def tokenize_function(inputs):
    # Tokenizar el batch completo
    model_inputs = tokenizer(
        inputs["text"], 
        max_length=1024, 
        truncation=True, 
        padding="max_length", 
        return_tensors="pt"
    )

    # Crear las labels como una copia de los input_ids
    labels = model_inputs["input_ids"].clone()

    # Obtener la secuencia de tokens para "answer:" (sin el token `<s>`)
    answer_colon_tokens = tokenizer("Answer:").input_ids[1:]  # Ahora solo [9412, 20]

    # Iterar sobre cada entrada en el batch
    for idx in range(labels.shape[0]):  
        input_ids = model_inputs["input_ids"][idx].tolist()  # Convertir a lista para iterar
        answer_start_idx = -1

        # Buscar la secuencia exacta "answer:" en input_ids
        for i in range(len(input_ids) - len(answer_colon_tokens) + 1):
            if input_ids[i : i + len(answer_colon_tokens)] == answer_colon_tokens:
                answer_start_idx = i + len(answer_colon_tokens)  # Inicio de la respuesta
                break

        if answer_start_idx != -1:
            labels[idx, :answer_start_idx] = -100  # Enmascarar todo antes de la respuesta
        else:
            labels[idx, :] = -100  # Si no se encuentra, enmascarar todo

    # Enmascarar también los tokens de padding
    labels[labels == tokenizer.pad_token_id] = -100

    return {"input_ids": model_inputs["input_ids"], "labels": labels}
def tokenize_function_test(inputs):
    model_inputs = tokenizer(inputs["text"], max_length=1024, padding_side='left',truncation=True,padding="max_length",return_tensors="pt")
    return model_inputs

def train_clean(X,Y):
    random.seed(1)
    # Agrupar ejemplos originales y sus copias
    train = X
    # Barajar los datos de entrenamiento para evitar sesgos
    train = train.sample(frac=1, random_state=1)
    train_y = Y.loc[train.index]
    train["label"]=train_y["label"]
    return train

def generative_test(X, Y, model):
    
    
        test=X
        test_y=Y.loc[train.index]
        test["label"]=test_y["label"]
        print(test)
        del test["text"]
        
        test.columns=["text","label"]
        print(test)
        
        # model = T5ForConditionalGeneration.from_pretrained(model_name)
        model=AutoModelForCausalLM.from_pretrained(f"results/{model_name}_second_stage_model_final")

        test_data=Dataset.from_pandas(test)
        test_data_all = test_data.map(tokenize_function_test, batched=True)
        test_data_all = test_data_all.remove_columns(["text","label"])
        test_data_all.set_format("torch")

    
        test_data={}
        test_label={}
        test_attention={}
        
        for tipo in questions:
            #Primero creamos la lista y luego vemos que no esté vacía para hacer el stack
            data=[output for j,output in enumerate(test_data_all["input_ids"].to("cuda")) if re.search(rf"_{tipo}(_\d+)?$", ind[j])]
            attention_masks=[output for j,output in enumerate(test_data_all["attention_mask"].to("cuda")) if re.search(rf"_{tipo}(_\d+)?$", ind[j])]
            
            if data:
                test_data[tipo]=torch.stack(data)
                test_attention[tipo]=torch.stack(attention_masks)
                del data
                del attention_masks

            labels=[output for j,output in enumerate(test["label"])  if re.search(rf"_{tipo}(_\d+)?$", ind[j])]
            if labels:
                test_label[tipo]=labels
        
        def generate_output(test_data, test_attention, tipo, tokens, beams):
            outputs = []
            for i,data in enumerate(test_data[tipo]):
                output = model.generate(
                    data.unsqueeze(0),  # Shape: (1, sequence_length)
                    max_new_tokens=tokens,
                    attention_mask=test_attention[tipo][i].unsqueeze(0),  # Important for reliable results

                    num_beams=beams,
                    num_return_sequences=1,
                    early_stopping=True
                ).squeeze(0).cpu()  # Shape: (generated_sequence_length,)
                
                outputs.append(output)  # Append 1D tensors (no extra dimensions)
        
            # Pad sequences to the longest one in the batch
            outputs_padded = pad_sequence(outputs, batch_first=True, padding_value=tokenizer.pad_token_id)
            return outputs_padded  # Shape: (batch_size, max_sequence_length)

            
            
        
        model=model.to("cuda")
        outputs={}
        
            
        if "age" in test_data:
            outputs["age"]=generate_output(test_data, test_attention,"age",tokens=2,beams=1)
            # outputs["age"]=generate_output(test_data,"age",tokens=10,beams=3)
            
                
        if "tipo" in test_data:
            # outputs["tipo"]=generate_output(test_data,"tipo",tokens=5,beams=1)
            if low_beams:
                outputs["tipo"]=generate_output(test_data, test_attention,"tipo",tokens=6,beams=1)
            else:
                outputs["tipo"]=generate_output(test_data, test_attention,"tipo",tokens=6,beams=5)
            

        if "tecnica" in test_data:
            if low_beams:
                outputs["tecnica"]=generate_output(test_data, test_attention,"tecnica",tokens=4,beams=1)
            else:
                
                outputs["tecnica"]=generate_output(test_data, test_attention,"tecnica",tokens=4,beams=5)
            # outputs["tecnica"]=generate_output(test_data,"tecnica",tokens=15,beams=3)
            
        if "history" in test_data:
            if low_beams:
                outputs["history"]=generate_output(test_data, test_attention,"history",tokens=29,beams=1)
            else:
                outputs["history"]=generate_output(test_data, test_attention,"history",tokens=29,beams=5)
            
        if "family" in test_data:
            if low_beams:
                outputs["family"]=generate_output(test_data, test_attention,"family",tokens=4,beams=1)
            else:
                outputs["family"]=generate_output(test_data, test_attention,"family",tokens=4,beams=5)
            # outputs["family"]=generate_output(test_data,"family",tokens=15,beams=3)
            
        if "symtomatic" in test_data:
            if low_beams:
                outputs["symtomatic"]=generate_output(test_data, test_attention,"symtomatic",tokens=7,beams=1)
            else:
                outputs["symtomatic"]=generate_output(test_data, test_attention,"symtomatic",tokens=7,beams=5)
            
        if "prosthesis" in test_data:
            outputs["prosthesis"]=generate_output(test_data, test_attention,"prosthesis",tokens=2,beams=1)
        if "birads" in test_data:
            if low_beams:
                outputs["birads"]=generate_output(test_data, test_attention,"birads",tokens=5,beams=1)
            else:
                outputs["birads"]=generate_output(test_data, test_attention,"birads",tokens=5,beams=5)
            # outputs["birads"]=generate_output(test_data,"birads",tokens=20,beams=3)
            
        if "density_mammo" in test_data:
            if low_beams:
                outputs["density_mammo"]=generate_output(test_data, test_attention,"density_mammo",tokens=2,beams=1)
            else:
                outputs["density_mammo"]=generate_output(test_data, test_attention,"density_mammo",tokens=2,beams=5)
        if "parenchymal_distortion" in test_data:
            if low_beams:
                outputs["parenchymal_distortion"]=generate_output(test_data, test_attention,"parenchymal_distortion",tokens=32,beams=1)
            else:
                outputs["parenchymal_distortion"]=generate_output(test_data, test_attention,"parenchymal_distortion",tokens=32,beams=5)

        if "calcifications_benign" in test_data:
            outputs["calcifications_benign"]=generate_output(test_data, test_attention,"calcifications_benign",tokens=2,beams=1)

        if "ganglio_mamo" in test_data:
            outputs["ganglio_mamo"]=generate_output(test_data, test_attention,"ganglio_mamo",tokens=2,beams=1)
            
        if "density_echo" in test_data:
            if low_beams:
                outputs["density_echo"]=generate_output(test_data, test_attention,"density_echo",tokens=5,beams=1)
            else:
                outputs["density_echo"]=generate_output(test_data, test_attention,"density_echo",tokens=5,beams=5)
            
        if "lymph_suspicious" in test_data:
            outputs["lymph_suspicious"]=generate_output(test_data, test_attention,"lymph_suspicious",tokens=2,beams=1)
        if "lymph_benign" in test_data:
            outputs["lymph_benign"]=generate_output(test_data, test_attention,"lymph_benign",tokens=2,beams=1)

        if "simple_cyst" in test_data:
            outputs["simple_cyst"]=generate_output(test_data, test_attention,"simple_cyst",tokens=2,beams=1)

        if "ductal_ectasia" in test_data:
            outputs["ductal_ectasia"]=generate_output(test_data, test_attention,"ductal_ectasia",tokens=2,beams=1)
            
        if "nodules_echo_num" in test_data:
            outputs["nodules_echo_num"]=generate_output(test_data, test_attention,"nodules_echo_num",tokens=2,beams=3)

        if "nodules_echo_description" in test_data:
            if low_beams:
                outputs["nodules_echo_description"]=generate_output(test_data, test_attention,"nodules_echo_description",tokens=60,beams=1)
                
            else:
                outputs["nodules_echo_description"]=generate_output(test_data, test_attention,"nodules_echo_description",tokens=60,beams=5)
            
        if "nodules_echo_shape" in test_data:
            if low_beams:
                outputs["nodules_echo_shape"]=generate_output(test_data, test_attention,"nodules_echo_shape",tokens=3,beams=1)
            else:
                outputs["nodules_echo_shape"]=generate_output(test_data, test_attention,"nodules_echo_shape",tokens=3,beams=3)

        if "nodules_echo_margin" in test_data:
            if low_beams:
                outputs["nodules_echo_margin"]=generate_output(test_data, test_attention,"nodules_echo_margin",tokens=4,beams=1)
            else:
                outputs["nodules_echo_margin"]=generate_output(test_data, test_attention,"nodules_echo_margin",tokens=4,beams=3)

        if "nodules_echo_echogenicity" in test_data:
            if low_beams:
                outputs["nodules_echo_echogenicity"]=generate_output(test_data, test_attention,"nodules_echo_echogenicity",tokens=5,beams=1)
            else:
                outputs["nodules_echo_echogenicity"]=generate_output(test_data, test_attention,"nodules_echo_echogenicity",tokens=5,beams=3)
        if "nodules_echo_location" in test_data:
            if low_beams:
                outputs["nodules_echo_location"]=generate_output(test_data, test_attention,"nodules_echo_location",tokens=15,beams=1)
            else:
                outputs["nodules_echo_location"]=generate_output(test_data, test_attention,"nodules_echo_location",tokens=15,beams=5)
        if "nodules_echo_size" in test_data:
            if low_beams:
                outputs["nodules_echo_size"]=generate_output(test_data, test_attention,"nodules_echo_size",tokens=7,beams=1)
            else:
                outputs["nodules_echo_size"]=generate_output(test_data, test_attention,"nodules_echo_size",tokens=7,beams=3)
            
        if "nodules_echo_known" in test_data:
            outputs["nodules_echo_known"]=generate_output(test_data, test_attention,"nodules_echo_known",tokens=2,beams=1)
            
        if "nodules_echo_stable" in test_data:
            if low_beams:
                outputs["nodules_echo_stable"]=generate_output(test_data, test_attention,"nodules_echo_stable",tokens=2,beams=1)
            else:
                outputs["nodules_echo_stable"]=generate_output(test_data, test_attention,"nodules_echo_stable",tokens=2,beams=3)
            

        text_out={}
        for tipo in questions:
            if tipo in outputs:
                text_out[tipo]=[re.search(r"Answer: (.+)", texto).group(1) if re.search(r"Answer: (.+)", texto) else "" for texto in tokenizer.batch_decode(outputs[tipo], skip_special_tokens=True)]
                print(f"{tipo} outputs")
                print(text_out[tipo])
                print(f"{tipo} labels")
                print(test_label[tipo])
                
        
        # # Mostrar los resultados finales
        # print(text_out_age)
        # print(test_label_age)
    
        # print(text_out_tipo)
        # print(test_label_tipo)
    
        # print(text_out_tecnica)
        # print(test_label_tecnica)

        # print(text_out_history)
        # print(test_label_history)
        
        
       
        # print(f"Accuracy: {accuracy:.2f}")
        
        ind_fold={tipo: [key for key in ind if re.search(rf"_{tipo}(_\d+)?$", key)] for tipo in questions}
        
        for tipo in questions:
            ind_cv[tipo].append(ind_fold[tipo])

        valid_dataset={tipo: [report for key,report in examples_raw.items() if key in ind_fold[tipo]] for tipo in questions }
        
    
        for tipo in questions:
            if tipo in text_out:
                print(f"{tipo} errors")
                # visualize_errors(valid_dataset[tipo],test_label[tipo],text_out[tipo],ind_fold[tipo])
            else:
                print(f"{tipo} does not appear in this fold")
        
    
    
                # Diccionario para almacenar accuracies por categoría
        
        # Calcular accuracies por categoría
        for tipo in questions:
            if tipo in text_out:
                acc = accuracy_score(test_label[tipo], text_out[tipo])
                accuracies_cv[tipo].append(acc)
                print(f"Accuracy for {tipo}: {acc:.4f}")
                preds_category_cv[tipo]+=text_out[tipo]
                labels_category_cv[tipo]+=test_label[tipo]
                output_dic_t[tipo]={ind:test_label[tipo][i] for i, ind in enumerate(ind_fold)}
                with open(f"truth_dic/{tipo}.pkl", "wb") as file:
                    pickle.dump(output_dic_t[tipo], file)
                output_dic[tipo]={ind:text_out[tipo][i] for i, ind in enumerate(ind_fold)}

        for tipo in questions:
            print(tipo)
            with open(f"results_dic_{tipo}/{model_name.split("/")[-1]}.pkl", "wb") as file:
                pickle.dump(output_dic[tipo], file)
                
        
